In [1]:
import cv2
import numpy as np
import subprocess
import os
from IPython.display import Image
import pandas as pd
import matplotlib.cm as cm
import matplotlib.pyplot as plt
%matplotlib inline
pd.options.display.max_columns = 120

# make sure we are at the top of the repo
wd = subprocess.check_output('git rev-parse --show-toplevel', shell = True)
os.chdir(wd[:-1]) #-1 removes \n

In [2]:
def obs_check(mask1, mask2, imgA, imgB, imgC):
    # generate the bikelane mask from our edited photos from each other
    mask1 = cv2.imread(mask1,0) # white
    mask2 = cv2.imread(mask2,0) # red
    mask = cv2.subtract(mask1, mask2) # red - white
   
    # write the mask to a jpg
    cv2.imwrite('pictures/mask.jpg', mask)
    
    # edit mask to give us lane identifier
    mask[mask < 10] = 0
    mask[mask >= 10] = 1

    # read in original images
    imgA = cv2.imread(imgA,0)
    imgB = cv2.imread(imgB,0)
    imgC = cv2.imread(imgC,0)
    
    # look only at the mask
    Aprime = mask * imgA
    Bprime = mask * imgB
    Cprime = mask * imgC
    
    # get diffs
    dAB = cv2.subtract(Bprime, Aprime)
    dBC = cv2.subtract(Cprime, Bprime)
    dAC = cv2.subtract(Cprime, Aprime)
    
    # export
    cv2.imwrite('pictures/aprime.jpg',Aprime)
    cv2.imwrite('pictures/bprime.jpg',Bprime)
    cv2.imwrite('pictures/cprime.jpg',Cprime)
    cv2.imwrite('pictures/dAB.jpg', dAB)
    cv2.imwrite('pictures/dBC.jpg', dBC)
    cv2.imwrite('pictures/dAC.jpg', dAC)
    
    if len(dAB[dAB > 50]) < 65:
        return(0)
    
    
    # use dAB as a mask
    #d = cv2.subtract(dBC, dAC)
    #cv2.imwrite('pictures/d.jpg', d)
    dAB[dAB < 20] = 0
    dAB[dAB >= 20] = 1
    
    #d[d < 20] = 0
    #d[d >= 20] = 1

    last = dAB * dBC
    cv2.imwrite('pictures/last.jpg', last)
    
    # threshold for changed pixels  
    if len(last[last > 20]) > 15:
        return(0)
    else:
        return(1)

In [3]:
# no obstruction case
obs_check('pictures/mask_1.jpg', 'pictures/mask_2.jpg', 'pictures/no_obs1.jpg', 'pictures/no_obs2.jpg', 'pictures/no_obs3.jpg')

0

In [4]:
# obstruction case
obs_check('pictures/mask_1.jpg', 'pictures/mask_2.jpg', 'pictures/obs1.jpg', 'pictures/obs2.jpg', 'pictures/obs3.jpg')

1

In [5]:
# no obstruction again
obs_check('pictures/mask_1.jpg', 'pictures/mask_2.jpg', 'pictures/Proj4_img00000011.jpg',
         'pictures/Proj4_img00000012.jpg', 'pictures/Proj4_img00000013.jpg')

0

In [6]:
# obstruction again
obs_check('pictures/mask_1.jpg', 'pictures/mask_2.jpg', 'pictures/Proj4_img00000071.jpg',
         'pictures/Proj4_img00000072.jpg', 'pictures/Proj4_img00000073.jpg')

1

In [7]:
# cyclists -- no obstruction
obs_check('pictures/mask_1.jpg', 'pictures/mask_2.jpg', 'pictures/Proj4_img00000019.jpg',
         'pictures/Proj4_img00000020.jpg', 'pictures/Proj4_img00000021.jpg')

0

In [8]:
# another obstruction
obs_check('pictures/mask_1.jpg', 'pictures/mask_2.jpg', 'pictures/Proj4_img00000172.jpg',
         'pictures/Proj4_img00000173.jpg', 'pictures/Proj4_img00000174.jpg')

1

In [9]:
dAB = cv2.imread('pictures/dAB.jpg',0)
len(dAB[dAB > 50])

68

In [10]:
last = cv2.imread('pictures/last.jpg',0)
len(last[last>20])

0

In [11]:
# for whole sampling period
pics = sorted(os.listdir('pictures/parkslope'))
obstructions = [obs_check('pictures/mask_1.jpg',
                         'pictures/mask_2.jpg',
                         'pictures/parkslope/'+pics[i],
                         'pictures/parkslope/'+pics[i+1],
                         'pictures/parkslope/'+pics[i+2]
                         ) for i in range(len(pics)-3)] 

In [12]:
zip(pics,obstructions)

[('Proj4_img00000001.jpg', 0),
 ('Proj4_img00000002.jpg', 0),
 ('Proj4_img00000003.jpg', 0),
 ('Proj4_img00000004.jpg', 1),
 ('Proj4_img00000005.jpg', 1),
 ('Proj4_img00000006.jpg', 0),
 ('Proj4_img00000007.jpg', 0),
 ('Proj4_img00000008.jpg', 0),
 ('Proj4_img00000009.jpg', 0),
 ('Proj4_img00000010.jpg', 0),
 ('Proj4_img00000011.jpg', 0),
 ('Proj4_img00000012.jpg', 0),
 ('Proj4_img00000013.jpg', 0),
 ('Proj4_img00000014.jpg', 0),
 ('Proj4_img00000015.jpg', 0),
 ('Proj4_img00000016.jpg', 0),
 ('Proj4_img00000017.jpg', 0),
 ('Proj4_img00000018.jpg', 0),
 ('Proj4_img00000019.jpg', 0),
 ('Proj4_img00000020.jpg', 1),
 ('Proj4_img00000021.jpg', 1),
 ('Proj4_img00000022.jpg', 1),
 ('Proj4_img00000023.jpg', 0),
 ('Proj4_img00000024.jpg', 0),
 ('Proj4_img00000025.jpg', 0),
 ('Proj4_img00000026.jpg', 0),
 ('Proj4_img00000027.jpg', 0),
 ('Proj4_img00000028.jpg', 0),
 ('Proj4_img00000029.jpg', 0),
 ('Proj4_img00000030.jpg', 0),
 ('Proj4_img00000031.jpg', 0),
 ('Proj4_img00000032.jpg', 0),
 ('Proj4

In [13]:
sum(obstructions)

29